I used the Reddit API, and the UCD subreddit.

For this project, we are going to collect data from Reddit, speciffically to investigate the UCD subreddit. We are choosing the praw wrapper to try to better learn how to use wrappers in calling to an api. We are going to invistigate some of the social dynamics of the UCD Subreddit. Collecting the data from UCD Subreddit using praw we collect both the top submissions to the UCD Subreddit, and all comments within those submissions. 

In [6]:
pip install praw

  Obtaining dependency information for praw from https://files.pythonhosted.org/packages/81/6a/21bc058bcccbe03f6a0895bf1bd60c805f0c526aa4e9bfaac775ed0b299c/praw-7.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for prawcore<3,>=2.1 from https://files.pythonhosted.org/packages/96/5c/8af904314e42d5401afcfaff69940dc448e974f80f7aa39b241a4fbf0cf1/prawcore-2.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 2.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
import praw
import requests
import csv
import pandas as pd
import os

In [25]:
cwd = os.getcwd()
print(cwd)

/Users/zach/Desktop/datasciencewithpython


Below are the different keys needed the api to allow for the accesses. We can cread an object reddit that has all the keys needed for the api retrieval. 

In [12]:
reddit = praw.Reddit(client_id= "######",
                     client_secret= "######",
                    user_agent = "#####",
                    username = "#####",
                    password = "######")

Using praw to retrieve submissions and comments, we will first get the submissions for the UCD subreddit. There are a few different ways of obtaining the submissions within the Praw wrapper: controversial, gilded, hot, new, rising, top. I chose top for this project. Setting the limit= None allows us to grab up to 1000 submissions to the subreddit. The time_filter is set to the past 'year'. We will collect the information of each submission by the id, title, author, time(created_utc), and comments retrieved. It is important that we set the final row in the variable of 'comments_retrieved' to be False, so that later on when we are collecting the data, we can update this paramter to true if we to retrieve the comment. This allows for when an error message can occur, we create a "save game" spot so when running through the loop, the code "knows where to go" and only make calls for comments with the comments_retrieved set to "false". We also need to add add a try block in case an "author's name is None" because of the error that can happen if the auhtor's name is missing. This appears to be because someone deleted their reddit account or no longer has a reddit account as it may have been removed/banned, but the post is still online.

The documentation that helped a lot with this is the praw documentation for the submission instance. Grabbing the "id" of the submissions is also quite important as it can be used later when grabbing the comments data. https://praw.readthedocs.io/en/stable/getting_started/quick_start.html

In [16]:
with open('./submissions.csv', 'w') as f:
    out = csv.writer(f)
    out.writerow(['id', 'title', 'author', 'created_utc', 'comments_retrieved'])
    for submission in reddit.subreddit('UCD').top(limit=None, time_filter = 'year'):
        try:
            name = submission.author.name
        except AttributeError:
            name = None
            print(submission)
        out.writerow([submission.id, submission.title, name, submission.created_utc, False])



11gyf77
10wtrb7
zt2gxl
zgd8dy
116gory
13j2389
11bp4rq
13qfeni
1300qud
114lr14
z9zgdw
170oye6
141icfm
13j6rs6
zuq0h7
z3qojz
yrc7pu
16bgy4y
10h3j7z


In [31]:
df = pd.read_csv('./submissions.csv')

In [34]:
df.head(5)

,id,title,author,created_utc,comments_retrieved
0,176x1xq,UCD bathrooms are wild,That_Bison_5628,1.697198e+09,True
1,16fc7qv,!! DOES ANYONE HAVE A FLOOR/MATTRESS I COULD S...,These-Occasion-1529,1.694381e+09,True
2,17hw6g0,How do I bring this up to my roomates?,filmfan4,1.698437e+09,True
3,17grx2v,Sick of exclusion and rude people,Curious_Bus2391,1.698310e+09,True
4,16mp12l,Thinking of dropping out,gotmyeyeonme,1.695125e+09,True


Because this call can be interupted and we can get an error message for making too many calls to the api, we want to begin with an if not, so that we do not keep rewriting over the same csv file. Here we are creating a csv and appending the information we are grabbing from the comments, which is the comments id, the body of the text, the author, the time, the "parent id"(who the comment was made to), the submission id (which submission is this part of), the amount of awards received for a comment, the upvotes the comment recieved, the down votes of the comment, as well as the overall "score" of the comment(how many total up and down votes it recieved). The amount of awards received, the upvotes and down votes, and the score may give insight into the comment, not just information regarding the network position of a comment/author.

Because we grabbed the submission id previously, we can use this to find all the comments within that submission. As the api returns a list via praw, again we are writing this to a csv. We use a lot of the documentation from praw and more information can be found here:
https://praw.readthedocs.io/en/stable/tutorials/comments.html

In [27]:
if not os.path.exists('./comments.csv'):
    with open('./comments.csv', 'w') as f:
        out = csv.writer(f)
        out.writerow(['id',
                      'body',
                      'author', 
                      'created_utc', 
                      'parent_id', 
                      'submission_id', 
                      'tot_awards_received', 
                      'ups', 
                      'downs', 
                      'score'])
# again, we set in the submission data for the "comments retrieved" to initially be false so we can locate the comments that haven't been retrieved.
for submission_id in df.loc[df.comments_retrieved == False, 'id']:
    # Creating this print command can allow us to track how many comments are being retrieved, as a progress check while the code is running
    print(f'Retrieving comments for {submission_id}')
    submission = reddit.submission(id=submission_id)
    # This sets the limit to None, which means that it will retrieve all comments, instead .
    submission.comments.replace_more(limit=None)
    # Because we're only storing whether a submission was retrieved, we save all the comments and write them at the same time in the curr comments.
    all_comments = []
    # We again need to add this in in case an "author's name is None" because of the error that can happen if someone no longer has a reddit accound, but the post is still online
    for comment in submission.comments.list():
        try:
            name = comment.author.name
        except AttributeError:
            name = None
        all_comments.append([comment.id, 
                        comment.body, 
                        name, 
                        comment.created_utc, 
                        comment.parent_id,
                        submission.id,
                        comment.total_awards_received,
                        comment.ups,
                        comment.downs,
                        comment.score
                        ])
    with open('./comments.csv', 'a') as f:
        out = csv.writer(f)
        out.writerows(curr_comments)
    # This allows us to change/update the paramater and create the "save game" situation in case the error message occurs and change the False to True
    df.loc[df.id == submission_id, 'comments_retrieved'] = True
    df.to_csv('./submissions.csv', index=False)
    
    

Retrieving comments for 15uqkwv
Retrieving comments for 15uko8s
Retrieving comments for 15ti0t0
Retrieving comments for 15n8dfz
Retrieving comments for 15n50s4
Retrieving comments for 15mp7k5
Retrieving comments for 15i0an3
Retrieving comments for 15gbyez
Retrieving comments for 15g2g90
Retrieving comments for 15ekj4v
Retrieving comments for 15dsfu0
Retrieving comments for 157clzd
Retrieving comments for 156ac38
Retrieving comments for 1560gla
Retrieving comments for 154mcal
Retrieving comments for 152w0q2
Retrieving comments for 151g0rb
Retrieving comments for 1515jql
Retrieving comments for 150lcnq
Retrieving comments for 14z6uop
Retrieving comments for 14x3qxc
Retrieving comments for 14ta0pv
Retrieving comments for 14nqk4d
Retrieving comments for 14nb4ml
Retrieving comments for 14mbufi
Retrieving comments for 14m72py
Retrieving comments for 14l7vfj
Retrieving comments for 14jt0fi
Retrieving comments for 14jjjye
Retrieving comments for 14hwwu6
Retrieving comments for 14hrh9c
Retrievi

Retrieving comments for 1615gm5
Retrieving comments for 16074g2
Retrieving comments for 15veb3t
Retrieving comments for 15uvzrm
Retrieving comments for 15uadbv
Retrieving comments for 15rrbw9
Retrieving comments for 15osom0
Retrieving comments for 15npna5
Retrieving comments for 15j0dw7
Retrieving comments for 15j05yi
Retrieving comments for 15h77r8
Retrieving comments for 15gas3f
Retrieving comments for 15fkp81
Retrieving comments for 15ecyz2
Retrieving comments for 15dv54t
Retrieving comments for 15a497y
Retrieving comments for 15a31rp
Retrieving comments for 158lgp4
Retrieving comments for 1540jsr
Retrieving comments for 153414c
Retrieving comments for 1532uh2
Retrieving comments for 152wq7x
Retrieving comments for 14ykvrx
Retrieving comments for 14xlbng
Retrieving comments for 14x218w
Retrieving comments for 14uw7i2
Retrieving comments for 14s26on
Retrieving comments for 14qgcpe
Retrieving comments for 14p5dql
Retrieving comments for 14o3vum
Retrieving comments for 14my41r
Retrievi

In [28]:
df1 = pd.read_csv('./comments.csv')

In [29]:
df1.head(5)

,id,body,author,created_utc,parent_id,submission_id,tot_awards_received,ups,downs,score
0,k4p2aih,Love how there's actual therapy going on in th...,seafoamteal,1.697199e+09,t3_176x1xq,176x1xq,0,48,0,48
1,k4pp7y3,I can’t see the mandatory; “All turds of mass ...,First-Can3099,1.697209e+09,t3_176x1xq,176x1xq,0,13,0,13
2,k4pjse8,Big difference to what was inscribed on the ba...,Michling99,1.697207e+09,t3_176x1xq,176x1xq,0,10,0,10
3,k4qbfye,Wow girls get words on the wall we get shit sm...,Verlop451,1.697217e+09,t3_176x1xq,176x1xq,0,8,0,8
4,k4qligd,The Newman toilets are next level. You always ...,LadyShadington,1.697221e+09,t3_176x1xq,176x1xq,0,9,0,9
